<a href="https://colab.research.google.com/github/MadeAsthito/Machine-Learning-Project/blob/main/classifier/gradient_boosting/cat_boost_implementation_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MACHINE LEARNING**


## **Analisis Klasifikasi Gaya Hidup**

---
Anggota Kelompok :
1. Kadek Cahyadi Yogachandra (2105551052)
2. I Kadek Krisna Apriana Dwi Guna (2105551065)
3. Made Asthito Yogi Prasanna (2105551078)
---
Ouput :

> 2 Model tertinggi berdasarkan data publik

- submission_normal_catboost.csv : 83,7%
  - Submisi Publik : 0.83706
  - Submisi Private : 0.85144
- submission_best_catboost.csv : 83,7%
  - Submisi Publik : 0.83466
  - Submisi Private : 0.85426

> Update untuk menyesuakan output berdasarkan leaderboards

- submission_most_optimum_catboost.csv :
  - Submisi Publik : 0.83666
  - Submisi Private : 0.85789

## **Import Library**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## **Import Dataset**
----
> Input file yang didapatkan dari kaggle kedalam session ini :
- train.csv
- test.csv

In [ ]:
df = pd.read_csv('./train.csv')
df

,id,umur,indeks_masa_badan,jenis_makanan,penyakit_khusus,penyakit_keturunan,lokasi_tinggal,mengatur_pola_makan,olahraga,jam_tidur_teratur,merokok,minum_alkohol,mengkonsumsi_suplemen,memperhatikan_kesehatan_mental,bersosialisasi,jumlah_sakit_setahun_terakhir,sehat
0,13777,51,29.45,jenis4,41,tidak ada,perkotaan,ya,tidak,ya,ya,tidak,tidak,tidak,tidak,2.0,tidak
1,17804,18,22.22,jenis2 jenis5,36,tidak ada,pedesaan,ya,tidak,tidak,tidak,tidak,tidak,tidak,ya,2.0,ya
2,2476,55,23.43,jenis3 jenis5,12,tidak ada,perkotaan,ya,ya,ya,tidak,ya,tidak,tidak,tidak,4.0,ya
3,18360,30,21.88,jenis3,40,tidak ada,perkotaan,ya,tidak,tidak,tidak,tidak,tidak,tidak,tidak,1.0,ya
4,17530,35,17.25,jenis6,33,tidak ada,pedesaan,ya,tidak,tidak,ya,tidak,ya,tidak,ya,3.0,tidak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19944,4821,26,21.37,jenis6,43,tidak ada,perkotaan,ya,tidak,tidak,tidak,tidak,tidak,tidak,tidak,1.0,ya
19945,21429,35,22.05,jenis5,28,tidak ada,perkotaan,ya,tidak,tidak,ya,tidak,tidak,tidak,ya,2.0,tidak
19946,18601,22,21.44,jenis6,49,tidak ada,pedesaan,ya,ya,tidak,ya,tidak,tidak,tidak,tidak,2.0,ya
19947,2067,13,21.66,jenis2 jenis3,12,tidak ada,pedesaan,ya,tidak,tidak,ya,tidak,tidak,tidak,tidak,1.0,ya


## **EDA**

### Cek Missing Value di dataset

In [ ]:
df.isnull().values.any()

False

### Cek Unique Value yang ada di dataset

In [ ]:
for column in df.columns:
    unique_values = df[column].unique()
    print(f"Column '{column}' unique values ({len(unique_values)}):")
    print(unique_values)
    print("\n" + "-"*40 + "\n")

Column 'id' unique values (19949):
[13777 17804  2476 ... 18601  2067 13485]

----------------------------------------

Column 'umur' unique values (67):
[51 18 55 30 35 57  1  3 49  8  6 29 46 21 48 56 61  7 17 16 10 20 39 26
  0 24 62 42 43 53 65 23 19 15 22 47  9  4 54 41 31 28 13 27 34 37 45 50
 32 12  5 52  2 14 63 58 64 44 40 60 25 11 36 33 38 66 59]

----------------------------------------

Column 'indeks_masa_badan' unique values (1301):
[29.45 22.22 23.43 ... 18.49 18.46 29.36]

----------------------------------------

Column 'jenis_makanan' unique values (32):
['jenis4 ' 'jenis2 jenis5 ' 'jenis3 jenis5 ' 'jenis3 ' 'jenis6' 'jenis5 '
 'jenis2 jenis3 ' 'jenis2 ' 'jenis1 ' 'jenis1 jenis4 jenis5 '
 'jenis3 jenis4 ' 'jenis1 jenis5 ' 'jenis1 jenis2 jenis3 '
 'jenis1 jenis2 jenis4 ' 'jenis1 jenis3 jenis5 ' 'jenis1 jenis3 '
 'jenis1 jenis2 jenis4 jenis5 ' 'jenis4 jenis5 ' 'jenis1 jenis2 '
 'jenis1 jenis2 jenis5 ' 'jenis2 jenis4 ' 'jenis1 jenis4 '
 'jenis2 jenis3 jenis5 ' 'jenis1 je

Hasil :
- Dataset tidak memiliki missing data
- Terdapat 3 kolom yang dapat di drop, yaitu
  - ID : Karena tidak membantu penilaian
  - penyakit_keturunan dan memperhatikan_kesehatan_mental : Karena hanya satu data saja
- Kolom jenis_makanan terdiri dari kompleks kategori, yang dapat dipecah dan diubah menjadi data biner (semacam onehotencoding)
- Terdapat proses mapping binary yang harus dilakukan untuk nilai :     
  - ya => 1
  - tidak => 0
  - perkotaan => 1
  - pedesaan => 0

## **Data Preprocessing**


---



> Kode program dibawah ini merupakan deklarasi fungsi `preprocessing`. Fungsi ini menerima sebuah DataFrame `df` dan parameter opsional `have_y` yang menentukan apakah label `y` ada dalam data. Fungsi ini pertama kali menginisialisasi variabel `y` sebagai `None` dan mendefinisikan peta biner untuk beberapa nilai string. Kemudian, beberapa kolom yang tidak diperlukan dihapus dari DataFrame. Berdasarkan nilai `have_y`, fungsi memisahkan fitur (X) dan label (y). Nilai dalam `X` kemudian diganti sesuai dengan peta biner.

> Kolom 'jenis_makanan' diolah dengan menghapus spasi di awal dan akhir string, kemudian diubah menjadi list string yang dipisahkan oleh spasi. Dengan menggunakan `MultiLabelBinarizer`, kolom ini diubah menjadi format one-hot encoding dan ditambahkan kembali ke DataFrame. Kolom asli 'jenis_makanan' kemudian dihapus. Jika `have_y` bernilai 1, label `y` dalam kolom 'sehat' diubah menjadi format biner menggunakan `LabelBinarizer`. Fungsi ini akan mengembalikan DataFrame fitur (X) dan label (y) jika ada.

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer, LabelBinarizer

def preprocessing(df, have_y = 0):
    y = None
    # Mapping binary
    binary_mapping = {'ya': 1, 'tidak': 0, 'pedesaan': 0, 'perkotaan': 1}

    # Menghapus kolom id karena tidak memberikan nilai
    # Menghapus kolom penyakit_keturunan dan memperhatikan_kesehatan_mental dikarenakan hanya memiliki satu nilai saja
    df = df.drop(['id', 'penyakit_keturunan', 'memperhatikan_kesehatan_mental'], axis=1)

    # Memisahkan fitur (X) dan label (y) jika have_y bernilai 1
    if have_y == 1:
        X = df.iloc[:, :-1]
    else:
        X = df.iloc[:, :]

    # Transformasi variabel X menjadi numerik dengan binary_mapping
    X = X.replace(binary_mapping)

    # Melakukan proses transformasi data jenis_makanan yang terpisah menjadi sebuah array
    X['jenis_makanan'] = X['jenis_makanan'].fillna('').astype(str)
    X['jenis_makanan'] = X['jenis_makanan'].str.strip()
    X['jenis_makanan'] = X['jenis_makanan'].apply(lambda x: x.split())

    # Inisialisasi MultiLabelBinarizer
    mlb = MultiLabelBinarizer()

    X_encoded = pd.DataFrame(mlb.fit_transform(X['jenis_makanan']), columns=mlb.classes_)
    X = pd.concat([X, X_encoded], axis=1)
    X.drop('jenis_makanan', axis=1, inplace=True)

    # Jika have_y bernilai 1, kolom label 'sehat' akan diubah menjadi biner
    if have_y == 1:
        lb = LabelBinarizer()
        y = df['sehat']
        y = lb.fit_transform(y)

    return X, y

## **Preprocess the Dataset**


---



> Kode program dibawah ini bertujuan untuk memanggil fungsi `preprocessing` dengan dua argumen: DataFrame `df` dan nilai `1` untuk parameter `have_y`. Dengan `have_y` diset ke `1`, fungsi akan menganggap bahwa DataFrame `df` memiliki kolom target yang perlu diproses. Hasil dari pemanggilan fungsi ini adalah dua variabel: `X`, yang berisi fitur-fitur yang telah diproses, dan `y`, yang berisi target yang telah dibinarisasi.

In [ ]:
X, y = preprocessing(df, 1)

In [ ]:
X

,umur,indeks_masa_badan,penyakit_khusus,lokasi_tinggal,mengatur_pola_makan,olahraga,jam_tidur_teratur,merokok,minum_alkohol,mengkonsumsi_suplemen,bersosialisasi,jumlah_sakit_setahun_terakhir,jenis1,jenis2,jenis3,jenis4,jenis5,jenis6
0,51,29.45,41,1,1,0,1,1,0,0,0,2.0,0,0,0,1,0,0
1,18,22.22,36,0,1,0,0,0,0,0,1,2.0,0,1,0,0,1,0
2,55,23.43,12,1,1,1,1,0,1,0,0,4.0,0,0,1,0,1,0
3,30,21.88,40,1,1,0,0,0,0,0,0,1.0,0,0,1,0,0,0
4,35,17.25,33,0,1,0,0,1,0,1,1,3.0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19944,26,21.37,43,1,1,0,0,0,0,0,0,1.0,0,0,0,0,0,1
19945,35,22.05,28,1,1,0,0,1,0,0,1,2.0,0,0,0,0,1,0
19946,22,21.44,49,0,1,1,0,1,0,0,0,2.0,0,0,0,0,0,1
19947,13,21.66,12,0,1,0,0,1,0,0,0,1.0,0,1,1,0,0,0


In [ ]:
y

array([[0],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

## **Catboost**

### Install

In [ ]:
# instalisasi library catboost
!pip install catboost

### Import Library

In [ ]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV

### Training Catboost Model


---



> Kode program dibawah ini bertujuan untuk membagi dataset menjadi data latih (`X_train`, `y_train`) dan data uji (`X_test`, `y_test`) menggunakan fungsi `train_test_split`, dengan 20% data digunakan sebagai data uji dan sisanya sebagai data latih. Model yang digunakan adalah `CatBoostClassifier` dari library CatBoost. Model ini diatur dengan berbagai parameter seperti jumlah iterasi, laju pembelajaran, kedalaman pohon, regulasi daun, fungsi kerugian, metrik evaluasi, penggunaan model terbaik, seed acak untuk reproducibility, dan tingkat verbosity.

> Setelah itu, model dilatih menggunakan data latih dan dievaluasi dengan data uji, sambil menampilkan plot proses pelatihan. Prediksi dibuat pada data uji, dan model dievaluasi menggunakan metrik akurasi (`accuracy_score`) dan AUC (Area Under the ROC Curve, dihitung dengan `roc_auc_score`). Hasil evaluasi kemudian dicetak, menunjukkan performa model dalam hal akurasi dan AUC.

In [ ]:
# Splitting dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Mengatur parameter catboost
model = CatBoostClassifier(iterations=1500,
                           learning_rate=0.01,
                           depth=7,
                           l2_leaf_reg=5,
                           loss_function='Logloss',
                           eval_metric='AUC',
                           use_best_model=True,
                           random_seed=42,
                           verbose=100)

# Training model
model.fit(X_train, y_train, eval_set=(X_test, y_test), plot=True)

# Membuat prediksi
y_pred = model.predict(X_test)

# Evaluasi model
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

print(f"Accuracy: {accuracy:.6f}")
print(f"AUC: {auc:.6f}")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.8259691	best: 0.8259691 (0)	total: 8.07ms	remaining: 12.1s
100:	test: 0.8857260	best: 0.8857260 (100)	total: 1.98s	remaining: 27.4s
200:	test: 0.9016772	best: 0.9016772 (200)	total: 4.32s	remaining: 27.9s
300:	test: 0.9092892	best: 0.9092892 (300)	total: 5.14s	remaining: 20.5s
400:	test: 0.9127112	best: 0.9127112 (400)	total: 6.92s	remaining: 19s
500:	test: 0.9147266	best: 0.9147373 (498)	total: 8.83s	remaining: 17.6s
600:	test: 0.9161031	best: 0.9161031 (600)	total: 10.7s	remaining: 16.1s
700:	test: 0.9171365	best: 0.9171365 (700)	total: 12.9s	remaining: 14.7s
800:	test: 0.9179092	best: 0.9179092 (800)	total: 16.3s	remaining: 14.2s
900:	test: 0.9184217	best: 0.9184235 (898)	total: 19.7s	remaining: 13.1s
1000:	test: 0.9186998	best: 0.9187358 (995)	total: 22s	remaining: 11s
1100:	test: 0.9190515	best: 0.9190515 (1100)	total: 24.7s	remaining: 8.96s
1200:	test: 0.9193187	best: 0.9193187 (1200)	total: 27.3s	remaining: 6.8s
1300:	test: 0.9195546	best: 0.9195912 (1290)	total: 31.6

### Training Catboost model dengan parameter paling optimal berdasarkan Leaderboards

> Update untuk menyesuaikan dengan output berdasarkan leaderboards

Note : Merupakan iterasi catboost dengan nilai benchmark private tertinggi

In [ ]:
# Splitting dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Mengatur parameter catboost
model_most_optimum = CatBoostClassifier(iterations=1000,
                           learning_rate=0.1,
                           depth=6,
                           l2_leaf_reg=5,
                           loss_function='Logloss',
                           eval_metric='AUC',
                           use_best_model=True,
                           random_seed=42,
                           verbose=100)

# Training model
model_most_optimum.fit(X_train, y_train, eval_set=(X_test, y_test), plot=True)

# Membuat prediksi
y_pred = model_most_optimum.predict(X_test)

# Evaluasi model
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, model_most_optimum.predict_proba(X_test)[:, 1])

print(f"Accuracy: {accuracy:.6f}")
print(f"AUC: {auc:.6f}")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.8157104	best: 0.8157104 (0)	total: 8.21ms	remaining: 8.21s
100:	test: 0.9187051	best: 0.9188626 (99)	total: 640ms	remaining: 5.69s
200:	test: 0.9209803	best: 0.9210345 (197)	total: 1.27s	remaining: 5.06s
300:	test: 0.9204267	best: 0.9210481 (201)	total: 1.9s	remaining: 4.42s
400:	test: 0.9196166	best: 0.9210481 (201)	total: 2.86s	remaining: 4.27s
500:	test: 0.9181233	best: 0.9210481 (201)	total: 4.4s	remaining: 4.38s
600:	test: 0.9176759	best: 0.9210481 (201)	total: 5.92s	remaining: 3.93s
700:	test: 0.9175345	best: 0.9210481 (201)	total: 6.59s	remaining: 2.81s
800:	test: 0.9166943	best: 0.9210481 (201)	total: 7.27s	remaining: 1.81s
900:	test: 0.9159072	best: 0.9210481 (201)	total: 7.88s	remaining: 866ms
999:	test: 0.9154740	best: 0.9210481 (201)	total: 8.57s	remaining: 0us

bestTest = 0.9210480802
bestIteration = 201

Shrink model to first 202 iterations.
Accuracy: 0.835840
AUC: 0.921048


### Tuning Hiperparameter dari Catboost

In [ ]:
param_space = {
    'learning_rate': [0.001, 0.01, 0.1, 0.5],
    'iterations': [900, 1000, 1500, 2000],
    'depth': [i for i in range(7,11,1)],
    'l2_leaf_reg': [i for i in range(4,7,1)],
    'loss_function': ['Logloss'],
    'eval_metric':['AUC'],
    'use_best_model': [True],
    'random_seed': [42],
    'verbose': [100]
}

# Mengatur parameter catboost
new_model = CatBoostClassifier()

# Perform random search cross-validation with 50 iterations
random_search = RandomizedSearchCV(estimator=new_model, param_distributions=param_space, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)

random_search.fit(X_train, y_train, eval_set=(X_test, y_test), plot=True)
# Membuat prediksi
y_pred = random_search.predict(X_test)

# Evaluasi model
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, random_search.predict_proba(X_test)[:, 1])

print(f"Accuracy: {accuracy:.6f}")
print(f"AUC: {auc:.6f}")

## **Prediction on Test Dataset**


---



> Proses selanjutnya yaitu melakukan prediksi pada data uji baru dengan menggunakan model Catboost yang sudah dilatih sebelumnya dengan melalui tahapan preprocessing yang sama.

### Import Dataset

In [ ]:
# Import data uji
df_test = pd.read_csv('./test.csv')
df_test

,id,umur,indeks_masa_badan,jenis_makanan,penyakit_khusus,penyakit_keturunan,lokasi_tinggal,mengatur_pola_makan,olahraga,jam_tidur_teratur,merokok,minum_alkohol,mengkonsumsi_suplemen,memperhatikan_kesehatan_mental,bersosialisasi,jumlah_sakit_setahun_terakhir
0,6774,5,18.03,jenis3 jenis4,26,tidak ada,pedesaan,ya,tidak,tidak,tidak,tidak,tidak,tidak,tidak,1.0
1,8117,34,28.07,jenis6,46,tidak ada,perkotaan,ya,tidak,tidak,ya,tidak,tidak,tidak,tidak,1.0
2,24780,52,23.71,jenis4,34,tidak ada,perkotaan,ya,tidak,tidak,ya,tidak,tidak,tidak,tidak,1.0
3,8532,16,22.93,jenis2,26,tidak ada,pedesaan,ya,ya,tidak,ya,tidak,tidak,tidak,ya,3.0
4,9139,3,29.44,jenis3,31,tidak ada,perkotaan,ya,tidak,tidak,tidak,tidak,tidak,tidak,tidak,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4983,9040,60,18.57,jenis6,15,tidak ada,pedesaan,tidak,ya,tidak,ya,tidak,tidak,tidak,tidak,1.0
4984,19022,64,29.18,jenis1,35,tidak ada,pedesaan,ya,tidak,ya,tidak,tidak,tidak,tidak,tidak,2.0
4985,2981,46,24.02,jenis2,25,tidak ada,perkotaan,ya,tidak,tidak,tidak,tidak,ya,tidak,ya,3.0
4986,15814,38,29.51,jenis3,11,tidak ada,pedesaan,ya,tidak,tidak,ya,tidak,tidak,tidak,tidak,1.0


### Preprocessing Dataset

In [ ]:
# Preprocessing data uji
X_pred, y_pred = preprocessing(df_test)

In [ ]:
X_pred

,umur,indeks_masa_badan,penyakit_khusus,lokasi_tinggal,mengatur_pola_makan,olahraga,jam_tidur_teratur,merokok,minum_alkohol,mengkonsumsi_suplemen,bersosialisasi,jumlah_sakit_setahun_terakhir,jenis1,jenis2,jenis3,jenis4,jenis5,jenis6
0,5,18.03,26,0,1,0,0,0,0,0,0,1.0,0,0,1,1,0,0
1,34,28.07,46,1,1,0,0,1,0,0,0,1.0,0,0,0,0,0,1
2,52,23.71,34,1,1,0,0,1,0,0,0,1.0,0,0,0,1,0,0
3,16,22.93,26,0,1,1,0,1,0,0,1,3.0,0,1,0,0,0,0
4,3,29.44,31,1,1,0,0,0,0,0,0,1.0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4983,60,18.57,15,0,0,1,0,1,0,0,0,1.0,0,0,0,0,0,1
4984,64,29.18,35,0,1,0,1,0,0,0,0,2.0,1,0,0,0,0,0
4985,46,24.02,25,1,1,0,0,0,0,1,1,3.0,0,1,0,0,0,0
4986,38,29.51,11,0,1,0,0,1,0,0,0,1.0,0,0,1,0,0,0


### Predicting the Dataset

In [ ]:
# Prediksi dengan model Catboost
y_pred = model.predict(X_pred)

In [ ]:
# Copy id pada data uji
df_subm = df_test[['id']].copy()

df_subm

,id
0,6774
1,8117
2,24780
3,8532
4,9139
...,...
4983,9040
4984,19022
4985,2981
4986,15814


> Kode program dibawah ini bertujuan untuk menambahkan prediksi hasil model `y_pred` ke dalam DataFrame `df_subm` pada kolom 'sehat'. Prediksi ini awalnya dalam bentuk nilai biner (0 atau 1). Untuk memudahkan interpretasi, nilai-nilai biner tersebut kemudian diganti dengan nilai string menggunakan dictionary `binary_mapping_sehat` yang memetakan 0 menjadi 'tidak' dan 1 menjadi 'ya'. Setelah penggantian ini, kolom 'sehat' dalam `df_subm` akan berisi nilai-nilai string yang lebih mudah dipahami.

In [ ]:
df_subm['sehat'] = y_pred

binary_mapping_sehat = {0: 'tidak', 1: 'ya'}
df_subm['sehat'] = df_subm['sehat'].replace(binary_mapping_sehat)

df_subm

,id,sehat
0,6774,tidak
1,8117,tidak
2,24780,tidak
3,8532,ya
4,9139,tidak
...,...,...
4983,9040,tidak
4984,19022,ya
4985,2981,ya
4986,15814,ya


## Export the result submission with .csv with normal Catboost

In [ ]:
df_subm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4988 entries, 0 to 4987
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      4988 non-null   int64 
 1   sehat   4988 non-null   object
dtypes: int64(1), object(1)
memory usage: 78.1+ KB


In [ ]:
# Simpan hasil prediksi ke submission.csv
df_subm.to_csv('submission_normal_catboost.csv', index=False)

## Export the result submission with .csv with best Catboost

In [ ]:
y_pred_best = random_search.predict(X_pred)

df_subm['sehat'] = y_pred_best

binary_mapping_sehat = {0: 'tidak', 1: 'ya'}
df_subm['sehat'] = df_subm['sehat'].replace(binary_mapping_sehat)

df_subm.to_csv('submission_best_catboost.csv', index=False)

## Export the result submission with .csv for the most optimal model based on private leaderboard

> Update untuk menyesuaikan dengan output berdasarkan leaderboards


In [ ]:
y_pred_optimum = model_most_optimum.predict(X_pred)

df_subm['sehat'] = y_pred_optimum

binary_mapping_sehat = {0: 'tidak', 1: 'ya'}
df_subm['sehat'] = df_subm['sehat'].replace(binary_mapping_sehat)

df_subm.to_csv('submission_most_optimum_catboost.csv', index=False)